<a href="https://colab.research.google.com/github/markste-in/collatz/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')


In [2]:
!lsblk

zsh:1: command not found: lsblk


In [3]:
!sudo file -s /dev/nvme1n1

Password:
sudo: a password is required


In [4]:
#!sudo mkfs -t xfs /dev/nvme1n1

In [5]:
!sudo mkdir /data
!sudo mount /dev/nvme1n1 /data
!sudo chown $USER:$USER /data

Password:
sudo: a password is required
Password:
sudo: a password is required

sudo: a password is required


In [1]:
from multiprocessing import Pool
import psutil
from helper import *


In [2]:
print("CPU Count:", psutil.cpu_count())

CPU Count: 8


In [3]:
if __name__ == '__main__':
    pool = Pool(psutil.cpu_count())
    path  = "~/collatz_dbs/"
    create_all(pool,path)




## Going from  3 to 500003 ##
Saved to ~/collatz_dbs/collatz_db_20210813-161440_741661059.parquet
Iteration took 7.617049932479858
## Going from  500003 to 1000003 ##
Saved to ~/collatz_dbs/collatz_db_20210813-161447_245902954.parquet
Iteration took 7.547245979309082
## Going from  1000003 to 1500003 ##
Saved to ~/collatz_dbs/collatz_db_20210813-161456_394028577.parquet
Iteration took 8.416938304901123
## Going from  1500003 to 2000003 ##
Saved to ~/collatz_dbs/collatz_db_20210813-161505_459973596.parquet
Iteration took 9.107331991195679
